In [ ]:
# Batch of imports
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from som import SOM

In [ ]:
input_dir = "./CharacterSets/Training"  # Training set directory
val_split = 0.2                         # Validation split
img_height = img_width = 50             # Height and width of the input images
batch_size = 100                        # Batch size

# Defining the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    input_dir,
    labels='inferred',
    validation_split=val_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)  # WTF does batch size affect????

# Defining the validation dataset
"""
val_ds = tf.keras.utils.image_dataset_from_directory(
    input_dir,
    labels='inferred',
    validation_split=val_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
"""


Notes:
train_ds upon initialization is a BatchDataset object
mnist in NN.py is a Module object
x_train is a complex numpy array

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
# print(class_names)

# Normalization layer. first_image can be viewed as a numpy array, but currently consists of all 1s for some reason.
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

"""
labels = np.array(image_batch, dtype=np.float32)
labels = np.reshape(labels, (-1, 1))

first_image = np.reshape(first_image, (-1,))
"""
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])
print(model)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


som = SOM(50, 50, 1, 0.5, 0.5, 100)
som.train(train_ds.take(10).as_numpy_iterator())


In [ ]:
# Trying to learn more about what train_ds actually is.
data = train_ds.take(100)
list(data.as_numpy_iterator())